In [ ]:
#%run tableau_export.py r85b548r prnzh4tp
%run tableau_export.py demo pr527dgq

Extracting Lumi data...
Creating doc table...


/usr/local/lib/python3.6/site-packages/se_code-0.1-py3.6.egg/se_code/fuzzy_logic.py:38: RuntimeWarning: invalid value encountered in double_scalars


Writing to file doc_table.csv.
Writing to file xref_table.csv.
Creating themes table...
Writing to file themes_table.csv.
Creating subset key terms table...


In [ ]:
%run tableau_export.py d53m338v prkpg9nt

In [ ]:
#SEPHORA
%run tableau_export.py h82y756m prdc54bx

In [ ]:
arr = [(1,1), (2,3), (3,2)]
arr = sorted(arr, key = lambda a:a[1])

In [ ]:
arr[-2:]

In [ ]:
from luminoso_api import LuminosoClient
from pack64 import unpack64
import run_voting_classifier # need accuracy/coverage chart
from conjunctions_disjunctions import get_new_results
from subset_key_terms import subset_key_terms
from scipy.stats import linregress

import csv
import json
import time
import sys
import datetime
import argparse
import numpy as np

In [ ]:
client, docs, topics, terms, subsets, drivers, skt, themes = pull_lumi_data('d53m338v', 'pr5cxsm3')

In [ ]:
def create_trends_table(terms, topics, docs):
    term_vecs = np.asarray([unpack64(t['vector']) for t in terms])
    concept_list = [t['text'] for t in terms]

    dated_docs = [d for d in docs if 'date' in d]
    dated_docs.sort(key = lambda k: k['date'])
    dates = np.asarray([[datetime.datetime.fromtimestamp(int(d['date'])).strftime('%Y-%m-%d %H:%M:%S')] for d in dated_docs])

    doc_vecs = np.asarray([unpack64(t['vector']) for t in dated_docs])
    results = np.dot(term_vecs, np.transpose(doc_vecs))
    results = np.transpose(results)
    idx = [[x] for x in range(0, len(results))]
    results = np.hstack((idx, results))
    
    headers = ['Date','Index']
    headers.extend(concept_list)
    
    tenth = int(.9 * len(results))
    quarter = int(.75 * len(results))
    half = int(.5 * len(results))

    slopes = [linregress(results[:,x+1],results[:,0])[0] for x in range(len(results[0])-1)]
    slope_ranking = zip(concept_list, slopes)
    slope_ranking = sorted(slope_ranking, key=lambda rank:rank[1])
    slope_ranking = slope_ranking[::-1]
    
    tenth_slopes = [linregress(results[tenth:,x+1],results[tenth:,0])[0] for x in range(len(results[0]) - 1)]
    tenth_slope_ranking = zip(concept_list, tenth_slopes)
    tenth_slope_ranking = sorted(tenth_slope_ranking, key=lambda rank:rank[1])
    tenth_slope_ranking = tenth_slope_ranking[::-1]
    
    quarter_slopes = [linregress(results[quarter:,x+1],results[quarter:,0])[0] for x in range(len(results[0]) - 1)]
    quarter_slope_ranking = zip(concept_list, quarter_slopes)
    quarter_slope_ranking = sorted(quarter_slope_ranking, key=lambda rank:rank[1])
    quarter_slope_ranking = quarter_slope_ranking[::-1]
    
    half_slopes = [linregress(results[half:,x+1],results[half:,0])[0] for x in range(len(results[0]) - 1)]
    half_slope_ranking = zip(concept_list, half_slopes)
    half_slope_ranking = sorted(half_slope_ranking, key=lambda rank:rank[1])
    half_slope_ranking = half_slope_ranking[::-1]
    
    results = np.hstack((dates, results))
    #trends_table = [{key:value for key, value in zip(headers, r)} for r in results]
    trends_table = []
    for r in results:
        dic = {}
        for key, value in zip(header, r):
            dic.update(key:value)
        trends_table.append(dic)
    trendingterms_table = [{'Term':term, 
                            'Slope':slope, 
                            'Rank':slope_ranking.index((term, slope)), 
                            'Short term slope':tenth_slope, 
                            'Short term rank':tenth_slope_ranking.index((term, tenth_slope)), 
                            'Medium term slope':quarter_slope,
                            'Medium term rank':quarter_slope_ranking.index((term, quarter_slope)), 
                            'Half term slope':half_slope, 
                            'Half term rank':half_slope_ranking.index((term, half_slope))}
                           for term, slope, tenth_slope, quarter_slope, half_slope in zip(concept_list, slopes, tenth_slopes, quarter_slopes, half_slopes)]

    return trends_table, trendingterms_table, results

#def create_prediction_table():
    
    
#def create_pairings_table():
    
    

In [ ]:
trends_table, trendingterms_table, results = create_trends_table(terms, topics, docs)